In [26]:
from tensorflow.keras.applications.mobilenet_v2 import preprocess_input
from tensorflow.keras.preprocessing.image import img_to_array
from tensorflow.keras.models import load_model
import numpy as np
import time
import random
import cv2
import os

## 1. Loading and configuration of model

In [27]:
dataset_dir = "./dataset/faces"
model_dir   = "./models"
model_name  = "/MobileNetV2.h5"

img_size = (160, 160)
color    = (255, 0, 0)

In [28]:
# Load the best saved model
from tensorflow.keras.models import load_model
model = load_model(model_dir + model_name)

In [29]:
# load our serialized face detector model from disk
prototxtPath = r"face_detector\deploy.prototxt"
weightsPath  = r"face_detector\res10_300x300_ssd_iter_140000.caffemodel"
faceNet = cv2.dnn.readNet(prototxtPath, weightsPath)

In [30]:
import json

class_labels = json.load(open('class_labels.json',))

def find_face_label(pred):
    index = 0
    valueMax = 0
    for i in range(len(pred)):
        if pred[i] > valueMax:
            valueMax = pred[i]
            index = i
    return get_key(index)
        
# function to return key for any value
def get_key(val):
    for key, value in class_labels.items():
         if val == value:
            return key

In [31]:
def predict_face(frame, faceNet, model):
    # grab the dimensions of the frame and then construct a blob
    # from it
    (h, w) = frame.shape[:2]
    blob = cv2.dnn.blobFromImage(
        frame, 
        1.0, 
        img_size,
        (104.0, 177.0, 123.0)
    )

    # pass the blob through the network and obtain the face detections
    faceNet.setInput(blob)
    detections = faceNet.forward()

    # initialize our list of faces, their corresponding locations,
    # and the list of predictions from our face mask network
    faces = []
    locs = []
    preds = []

    # loop over the detections
    for i in range(0, detections.shape[2]):
        # extract the confidence (i.e., probability) associated with
        # the detection
        confidence = detections[0, 0, i, 2]

        # filter out weak detections by ensuring the confidence is
        # greater than the minimum confidence
        if confidence > 0.5:
            # compute the (x, y)-coordinates of the bounding box for
            # the object
            box = detections[0, 0, i, 3:7] * np.array([w, h, w, h])
            (startX, startY, endX, endY) = box.astype("int")

            # ensure the bounding boxes fall within the dimensions of
            # the frame
            (startX, startY) = (max(0, startX), max(0, startY))
            (endX, endY) = (min(w - 1, endX), min(h - 1, endY))

            # extract the face ROI, convert it from BGR to RGB channel
            # ordering, resize it to 160x160, and preprocess it
            face = frame[startY:endY, startX:endX]
            face = cv2.cvtColor(face, cv2.COLOR_BGR2RGB)
            face = cv2.resize(face, img_size)
            face = img_to_array(face)
            face = preprocess_input(face)

            # add the face and bounding boxes to their respective
            # lists
            faces.append(face)
            locs.append((startX, startY, endX, endY))

    # only make a predictions if at least one face was detected
    if len(faces) > 0:
        # for faster inference we'll make batch predictions on *all*
        # faces at the same time rather than one-by-one predictions
        # in the above `for` loop
        faces = np.array(faces, dtype="float32")
        preds = model.predict(faces, batch_size=32)

    # return a 2-tuple of the face locations and their corresponding
    # locations
    return (locs, preds)

## 2. Predicting the classification of webcam

In [ ]:
# initialize the video stream
cap = cv2.VideoCapture(0)

# loop over the frames from the video stream
while True:
    # grab the frame from the threaded video stream and resize it
    # to have a maximum width of 400 pixels
    success, frame = cap.read()
    
    # detect faces in the frame and determine if they are wearing a
    # face mask or not
    (locs, preds) = predict_face(frame, faceNet, model)

    # loop over the detected face locations and their corresponding
    # locations
    for (box, pred) in zip(locs, preds):
        # unpack the bounding box and predictions
        (startX, startY, endX, endY) = box
        
        label = find_face_label(pred)

        # include the probability in the label
        label = "{}: {:.2f}%".format(label, max(pred[0], pred[1]) * 100)

        # display the label and bounding box rectangle on the output
        # frame
        cv2.putText(frame, label, (startX, startY - 10),
            cv2.FONT_HERSHEY_SIMPLEX, 0.95, color, 2)
        cv2.rectangle(frame, (startX, startY), (endX, endY), color, 2)

    # show the output frame
    cv2.imshow("Frame", frame)
    key = cv2.waitKey(1) & 0xFF

    # if the `q` key was pressed, break from the loop
    if key == ord("q"):
        break

# do a bit of cleanup
cv2.destroyAllWindows()
vs.stop()